In [1]:
import pandas as pd
import numpy as np
from collections import defaultdict
import pickle
from sklearn.cluster import KMeans
import warnings
warnings.filterwarnings('ignore')
from sklearn.utils import shuffle


In [3]:
#dataset_name = 'datasets/spambase.csv'
dataset_name = 'C:/Users/Vishal/Downloads/datasets/datasets/spambase.csv'
data = pd.read_csv(dataset_name)
THRESHOLD=0.95
Q=5
BETA=0.1
pool={}
# data = data.sample(frac=1).reset_index(drop=True)
# data = data.apply(pd.to_numeric).fillna(0)
# data = data.iloc[:, 1:]
data

,X1,X2,X3,X4,X5,X6,X7,X8,X9,X10,...,X49,X50,X51,X52,X53,X54,X55,X56,X57,Y
0,0.00,0.00,0.00,0.0,0.00,0.00,0.0,0.00,0.00,0.00,...,0.000,0.000,0.0,0.000,0.000,0.000,1.214,4,17,0
1,0.05,0.30,0.40,0.0,0.10,0.05,0.0,0.05,0.10,0.00,...,0.000,0.036,0.0,0.054,0.118,0.000,2.379,96,583,1
2,0.00,0.76,0.00,0.0,0.00,0.00,0.0,0.00,0.57,0.19,...,0.000,0.099,0.0,0.232,0.066,0.928,20.432,213,1655,1
3,0.09,0.49,0.59,0.0,0.39,0.19,0.0,0.00,0.09,0.39,...,0.765,0.037,0.0,5.828,1.308,0.000,6.047,54,768,1
4,0.00,0.00,0.78,0.0,0.00,0.00,0.0,0.00,0.00,0.00,...,0.000,0.401,0.0,0.133,0.000,0.000,1.565,4,36,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4596,0.00,0.00,0.00,0.0,0.00,0.00,0.0,0.00,0.00,0.00,...,0.000,0.000,0.0,0.000,0.000,0.000,1.000,1,5,0
4597,0.00,0.00,0.00,0.0,0.00,0.00,0.0,0.00,0.00,0.00,...,0.000,0.000,0.0,0.000,0.000,0.000,1.000,1,4,0
4598,0.00,0.00,0.00,0.0,0.00,0.00,0.0,0.00,0.00,0.00,...,0.000,0.000,0.0,0.000,0.000,0.000,1.500,9,30,0
4599,0.00,0.00,0.26,0.0,0.00,0.00,0.0,0.00,0.00,0.00,...,0.036,0.109,0.0,0.000,0.036,0.000,1.632,11,307,0


In [5]:

import matplotlib.pyplot as plt
from matplotlib import style

style.use('ggplot')
import numpy as np

colors = 10 * ["g", "r", "c", "b", "k"]


class K_Means:
    def __init__(self, k=5, tol=0.001, max_iter=300):
        self.k = k
        self.tol = tol
        self.max_iter = max_iter

    '''def find_second_smallest(self, numbers):
        m1, m2 = float('inf'), float('inf')
        for x in numbers:
            if x <= m1:
                m1, m2 = x, m1
            elif x < m2:
                m2 = x
        return m2'''
    
    def find_max_labels_in_cluster(self, cluster):
        #print("Cluster: " + str(cluster))
        labels = []
        for arr in cluster:
            labels.append(arr[-1])
        return max(labels, key=labels.count)

    def fit(self, data):

        self.centroids = {}
        self.max_labels = {}

        for i in range(self.k):
            self.centroids[i] = data[i][:-1]
            #print(self.centroids[i].shape)
            self.max_labels[i] = -9

        for i in range(self.max_iter):
            self.classifications = {}

            for i in range(self.k):
                self.classifications[i] = []
            
            for featureset in data:
                #print(self.max_iter,self.max_labels)
                distances = [np.linalg.norm(featureset[:-1] - self.centroids[centroid]) for centroid in self.centroids]#Euclidean distance                
                classification = distances.index(min(distances))#Initial cluster
                
                if(featureset[-1] != -99):#Has a label
                    if(self.max_labels[classification] == -9):#No labelled data is assigned to the cluster yet
                        self.classifications[classification].append(featureset)
                        self.max_labels[classification] = featureset[-1]
                    elif(self.max_labels[classification] == featureset[-1]):#Max label matches then assign to that cluster
                        self.classifications[classification].append(featureset)
                        self.max_labels[classification] = self.find_max_labels_in_cluster(self.classifications[classification])
                    else:#Assign to second nearst cluster with label equal to featureset[-1]
                        labels_match = {}
                        matched_centroids = {}
                        
                        for m in self.max_labels:
                            if self.max_labels[m] == featureset[-1]:
                                labels_match[m] = self.max_labels[m]
                                matched_centroids[m] = self.centroids[m]
                        
                        if len(labels_match) == 0:#Empty. Assign to nearest cluster
                            self.classifications[classification].append(featureset)
                            self.max_labels[classification] = self.find_max_labels_in_cluster(self.classifications[classification])
                        else:
                            distances1 = [np.linalg.norm(featureset[:-1] - matched_centroids[centroid]) for centroid in matched_centroids]#Euclidean distance
                            classification = distances1.index(min(distances1))
                            self.classifications[classification].append(featureset)
                            self.max_labels[classification] = self.find_max_labels_in_cluster(self.classifications[classification])

                else:#Unlabelled
                    self.classifications[classification].append(featureset)
            print("MAX LABELS: " + str(self.max_labels))
            #print("Classifications: " + str(self.classifications))

            prev_centroids = dict(self.centroids)
            #print("AAAA")
            
                
            for classification in self.classifications:
                self.centroids[classification] = np.average(self.classifications[classification], axis=0)
                self.centroids[classification]=np.delete(self.centroids[classification],-1)
            optimized = True

            for c in self.centroids:
                
                #print("AAA")
                original_centroid = prev_centroids[c]
                current_centroid = self.centroids[c]
                #print(original_centroid.shape)
                #print(current_centroid.shape)
                if np.sum((current_centroid - original_centroid) / original_centroid * 100.0) > self.tol:
                    optimized = False

            if optimized:
                break

    def predict_first(self, data,mapping):
        distances = [np.linalg.norm(data - self.centroids[centroid][:-1]) for centroid in self.centroids]
        classification = distances.index(min(distances))
        return classification
    
    def predict(self,data,mapping):
        #print(data.shape)
        #print(self.centroids[0].shape)
        dist={}
        for x in mapping.keys():
            if(mapping[x]!=-9):
                dist[x]=np.linalg.norm(data - self.centroids[x])
    #if (len(sorted(dist.items(), key = lambda kv:(kv[1], kv[0])))==1):
        #return sorted(dist.items(), key = lambda kv:(kv[1], kv[0]))[0]
        return sorted(dist.items(), key = lambda kv:(kv[1], kv[0]))[0]
    def map_funct(self):
        return self.max_labels
    def update(self, new_data, delta):
        
        for featureset in new_data:
            
            distances = [np.linalg.norm(featureset - self.centroids[centroid]) for centroid in self.centroids]
            classification = distances.index(min(distances))
            self.classifications[classification].append(featureset)
            self.centroids[classification] = np.average(self.classifications[classification], axis=0)

In [6]:
def create_classifier(data,Q,batch_split):
    #print(data)
    full_batch=data.iloc[:,:]
    full_batch.iloc[batch_split:, -1] = -99 #Mark unlabelled data's label as "-1"
    #print(full_batch)
    kmeans = K_Means()
    kmeans.fit(np.array(np.array(full_batch)))
    mapping=kmeans.map_funct()
    '''labelled_batch=data.iloc[:batch_split,:]
    pred=[]
    for i in range(len(labelled_batch)):
        pred.append(kmeans.predict_first(np.array(data.iloc[0:len(labelled_batch),:-1])[i]))
    labelled_batch['pred']=pred
    df=labelled_batch.iloc[:,-2:]
    a=df.groupby(['pred']).agg(lambda x:x.value_counts().index[0])
    mapping={}
    b=a.reset_index()
    for i in range(0,len(b['pred'])):
        mapping[b['pred'][i]]=b.iloc[:,-1][i]
    print(mapping)'''
    return kmeans.centroids,mapping,kmeans.classifications

In [7]:
def update_pool(labelled_batch,batch_no,full_batch):
    batch_simi={}
    for classifier_name in pool:
        similarity=find_accuracy(classifier_name,labelled_batch)
        batch_simi[classifier_name]=similarity
        
    max_similarity_key=max(batch_simi, key=batch_simi.get)
    max_similarity = batch_simi[max_similarity_key]
    print(batch_simi)
    model,mapping,classes=create_classifier(batch,Q,batch_split)
    if(max_similarity>THRESHOLD or len(pool)>=max_c):
        model_file=open(max_similarity_key,'wb')
        map_file=open(max_similarity_key.split(".")[0],'wb')
        no=str(str(max_similarity_key.split("Batch")[1]).split(".")[0])
        class_file=open("Class"+str(no),'wb')
        pickle.dump(model,model_file)
        pickle.dump(mapping,map_file)
        pickle.dump(classes,class_file)
        pool[max_similarity_key]=1
        '''model_file=open(max_similarity_key,'rb')
        map_file=open(max_similarity_key.split(".")[0],'rb')
        no=str(str(max_similarity_key.split("Batch")[1]).split(".")[0])
        class_file=open("Class"+str(no),'rb')
        km=K_Means()
        km.centroids=pickle.load(model_file)
        km.classifications=pickle.load(class_file)
        km.update(labelled_batch.iloc[:,-1],9)
        model_file=open(max_similarity_key,'wb')
        class_file=open("Class"+str(no),'wb')
        pickle.dump(km.classifications,class_file)
        pickle.dump(km.centroids,model_file)'''
        
    else:
        model_name="Batch"+str(batch_no)+".sav"
        map_name="Batch"+str(batch_no)
        class_name="Class"+str(batch_no)
        model_file=open(model_name,'wb')
        map_file=open(map_name,'wb')
        class_file=open(class_name,'wb')
        pickle.dump(model,model_file)
        pickle.dump(mapping,map_file)
        pickle.dump(classes,class_file)
        pool[model_name]=1
    return pool

In [8]:
def find_accuracy(classifier_name,labelled_batch):
    s=0
    model_file=open(classifier_name,'rb')
    map_file=open(classifier_name.split(".")[0],'rb')
    model=pickle.load(model_file)
    mapping=pickle.load(map_file)
    no=str(str(classifier_name.split("Batch")[1]).split(".")[0])
    class_file=open("Class"+str(no),'rb')
    classes=pickle.load(class_file)
    km=K_Means()
    km.classifications=classes
    km.centroids=model
    for i in range(len(labelled_batch)):
        current_row=labelled_batch.iloc[i:i+1,:-1]
        current_label=labelled_batch.iloc[i:i+1,-1]
        pred=km.predict(current_row,mapping)
        final_pred=mapping[pred[0]]
        if(final_pred==current_label[i]):
            s+=1
    return((s)/len(labelled_batch))

In [9]:
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score,precision_recall_fscore_support,roc_auc_score, accuracy_score, f1_score,recall_score,precision_score

In [13]:
pool={}
batch_no=1
stream_size=int(100)
skip=0
batch_split = int(0.2 * stream_size)
max_c=10
acc=[]
s=0
full_pred=[]
full_true=[]
final_df = pd.DataFrame(columns = ['Dataset', 'Labelled percent', 'Distance used','Batch No',  'Accuracy', 'F1', 'Recall', 'Precision', 'Report'])
#for j in range(0,6):
for j in range(0,int(len(data)/stream_size)):
    print("Batch",batch_no)
    batch = pd.read_csv(dataset_name, nrows = stream_size, skiprows=skip)
    
    if(j==0):
        model_name="Batch"+str(batch_no)+".sav"#Initial classifier
        map_name="Batch"+str(batch_no)
        class_name="Class"+str(batch_no)
        model,mapping,classes=create_classifier(batch,Q,batch_split)
        pool[model_name]=1
        model_file=open(model_name,'wb')
        map_file=open(map_name,'wb')
        class_file=open(class_name,'wb')
        pickle.dump(model,model_file)
        pickle.dump(mapping,map_file)
        pickle.dump(classes,class_file)
    else:
        #acc=[]
        s=0
        labelled_batch=batch.iloc[:batch_split,:]
        pred_unlabelled=[]
        true_unlabelled=[]
        for i in range(len(labelled_batch)):
            current_row=labelled_batch.iloc[i:i+1,:-1]
            current_label=labelled_batch.iloc[i:i+1,-1]
            Keymax = max(pool, key=pool.get)
            model_file=open(Keymax,'rb')
            map_file=open(Keymax.split(".")[0],'rb')
            no=str(str(Keymax.split("Batch")[1]).split(".")[0])
            class_file=open("Class"+str(no),'rb')
            model=pickle.load(model_file)
            mapping=pickle.load(map_file)
            classes=pickle.load(class_file)
            km=K_Means()
            km.centroids=model
            km.classifications=classes
            pred=km.predict(current_row,mapping)
            final_pred=mapping[pred[0]]
            pred_unlabelled.append(final_pred)
            true_unlabelled.append(current_label[i])
            full_pred.append(final_pred)
            full_true.append(current_label[i])
            for classifier_name in pool:            
                model_file=open(classifier_name,'rb')
                map_file=open(classifier_name.split(".")[0],'rb')
                no=str(str(classifier_name.split("Batch")[1]).split(".")[0])
                class_file=open("Class"+str(no),'rb')
                model=pickle.load(model_file)
                mapping=pickle.load(map_file)
                classes=pickle.load(class_file)
                km=K_Means()
                km.centroids=model
                km.classifications=classes
                pred=km.predict(current_row,mapping)
                final_pred=mapping[pred[0]]
                if(final_pred!=current_label[i]):
                    pool[classifier_name]=pool[classifier_name]*(BETA**1)#Reduce weight otherwise
        unlabelled_batch=batch.iloc[batch_split:,:]
        for i in range(len(unlabelled_batch)):
            current_row=unlabelled_batch.iloc[i:i+1,:-1]
            current_label=unlabelled_batch.iloc[i:i+1,-1]
            Keymax = max(pool, key=pool.get)
            model_file=open(Keymax,'rb')
            map_file=open(Keymax.split(".")[0],'rb')
            no=str(str(Keymax.split("Batch")[1]).split(".")[0])
            class_file=open("Class"+str(no),'rb')
            model=pickle.load(model_file)
            mapping=pickle.load(map_file)
            classes=pickle.load(class_file)
            km=K_Means()
            km.centroids=model
            km.classifications=classes
            pred=km.predict(current_row,mapping)
            final_pred=mapping[pred[0]]
            pred_unlabelled.append(final_pred)
            true_unlabelled.append(current_label[i+batch_split])
            full_pred.append(final_pred)
            full_true.append(current_label[i+batch_split])
            #if(final_pred==current_label[i+batch_split]):
              #  s+=1
        pool=update_pool(labelled_batch,batch_no,batch)#Update pool
        #print(true_unlabelled)
        #print(pred_unlabelled)
        acc = accuracy_score(true_unlabelled,pred_unlabelled)
        f1 = f1_score(true_unlabelled,pred_unlabelled,average=None)
        re = recall_score(true_unlabelled, pred_unlabelled, average = None)
        pre = precision_score(true_unlabelled, pred_unlabelled, average = None)
        rep = classification_report(true_unlabelled, pred_unlabelled)

        final_df.loc[len(final_df)] = {'Dataset':"Spambase", 'Labelled percent':'20',
                                    'Distance used':'Euclidean','Batch No': batch_no, 'Accuracy': acc, 'F1': f1,
                                    'Recall': re, 'Precision': pre, 'Report': rep} 
        print(classification_report(true_unlabelled,pred_unlabelled))
    #print(acc)
    skip += stream_size
    batch_no += 1
acc1 = accuracy_score(full_true,full_pred)
f11 = f1_score(full_true,full_pred,average=None)
re1 = recall_score(full_true, full_pred, average = None)
pre1 = precision_score(full_true, full_pred, average = None)
rep1 = classification_report(full_true, full_pred)

final_df.loc[len(final_df)] = {'Dataset':"Spambase", 'Labelled percent':'20',
                                    'Distance used':'Euclidean','Batch No': 'Full', 'Accuracy': acc1, 'F1': f11,
                                    'Recall': re1, 'Precision': pre1, 'Report': rep1} 
final_df.to_csv('SPASC_results_' + 'Spambase' +  '_ossilh.csv')

Batch 1
MAX LABELS: {0: 0.0, 1: 1.0, 2: 1.0, 3: 1.0, 4: 0.0}
Batch 2
{'Batch1.sav': 0.75}
MAX LABELS: {0: 1.0, 1: 1.0, 2: 1.0, 3: 0.0, 4: 0.0}
              precision    recall  f1-score   support

           0       0.64      0.91      0.75        54
           1       0.79      0.41      0.54        46

    accuracy                           0.68       100
   macro avg       0.72      0.66      0.65       100
weighted avg       0.71      0.68      0.66       100

Batch 3
{'Batch1.sav': 0.75, 'Batch2.sav': 0.65}
MAX LABELS: {0: 1.0, 1: 1.0, 2: 0.0, 3: 0.0, 4: 0.0}
              precision    recall  f1-score   support

           0       0.56      0.73      0.63        52
           1       0.56      0.38      0.45        48

    accuracy                           0.56       100
   macro avg       0.56      0.55      0.54       100
weighted avg       0.56      0.56      0.55       100

Batch 4


KeyboardInterrupt: 